<a href="https://colab.research.google.com/github/kiansiong/exploratory_projects/blob/master/DeepLearning1_MalwareAnalysis_CNN_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from os import listdir
import matplotlib.pyplot as plt
from datetime import datetime
from pytz import timezone

# import tensorflow as tf
from keras import metrics
from keras.optimizers import Adam, SGD
from keras.models import Sequential, load_model, Model
from keras import backend
from keras.layers import GRU, Conv1D, Dense, AveragePooling1D, MaxPooling1D, Dropout, Input, concatenate, LSTM
from keras.callbacks import EarlyStopping, Callback, LearningRateScheduler, TensorBoard, ReduceLROnPlateau, ModelCheckpoint, History
from keras import regularizers

Using TensorFlow backend.


In [0]:
#zip archive !need to check why this does work
zipFile = "/content/drive/My Drive/NUS_2019/cs5242_kaggle/data/cs5242-neural-networks-and-deep-learning.zip"

#file paths
dataPath_train = '/content/kaggle/train/'
dataPath_test = '/content/kaggle/test/'
target = '/content/kaggle/train_kaggle.csv'

In [0]:
#mount drive to access files. if files are deleted, have to unzip from 'My Drive' again. 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Environment Setup

In [0]:
!mkdir /content/kaggle
!ls "/content/drive/My Drive/NUS_2019/cs5242_kaggle/data"

mkdir: cannot create directory ‘/content/kaggle’: File exists
cs5242-neural-networks-and-deep-learning.zip


**Manually download zip file from Kaggle, and upload to My Drive. Then unzip**

In [0]:
%%capture 
!unzip "/content/drive/My Drive/NUS_2019/cs5242_kaggle/data/cs5242-neural-networks-and-deep-learning.zip" -d "/content/kaggle"

In [0]:
#move unzipped files to working directory
!mv /content/kaggle/train/train/*.npy /content/kaggle/train
!mv /content/kaggle/test/test/*.npy /content/kaggle/test

mv: cannot stat '/content/kaggle/train/train/*.npy': No such file or directory
mv: cannot stat '/content/kaggle/test/test/*.npy': No such file or directory


In [0]:
#remove empty folder
!rmdir /content/kaggle/train/train
!rmdir /content/kaggle/test/test

rmdir: failed to remove '/content/kaggle/train/train': No such file or directory
rmdir: failed to remove '/content/kaggle/test/test': No such file or directory


In [0]:
#check file count is correct. train=18662, test=6051
!ls /content/kaggle/train | wc -l
!ls /content/kaggle/test | wc -l

18662
6051


In [0]:
#top 5 training files
!ls -lrt /content/kaggle/train | head -5

#top 5 test files
!ls -lrt /content/kaggle/test | head -5

total 4344112
-rw-r--r-- 1 root root  43376 Sep 10 05:00 1.npy
-rw-r--r-- 1 root root 408128 Sep 10 05:00 10.npy
-rw-r--r-- 1 root root 408128 Sep 10 05:00 101.npy
-rw-r--r-- 1 root root 408128 Sep 10 05:00 1010.npy
total 1071112
-rw-r--r-- 1 root root 408128 Sep 10 04:58 1.npy
-rw-r--r-- 1 root root  30728 Sep 10 04:58 10.npy
-rw-r--r-- 1 root root 408128 Sep 10 04:58 100.npy
-rw-r--r-- 1 root root 408128 Sep 10 04:58 1005.npy


# Initialise and Check Data
Also to check if data unzipped correctly

In [0]:
# data = pd.read_csv(target, header=0, sep=',', quotechar='"')
# data.head()

In [0]:
!ls -lrt /content/kaggle/train | head -5

total 4344112
-rw-r--r-- 1 root root  43376 Sep 10 05:00 1.npy
-rw-r--r-- 1 root root 408128 Sep 10 05:00 10.npy
-rw-r--r-- 1 root root 408128 Sep 10 05:00 101.npy
-rw-r--r-- 1 root root 408128 Sep 10 05:00 1010.npy


In [0]:
# test_load = np.load(dataPath_train+'1.npy')
# print(test_load.shape)

In [0]:
# test_load[:5,:5]

In [0]:
# # to load numpy file type
# sampleTrain = np.load(dataPath_train + '11.npy')
# sampleTrain.shape

In [0]:
# read all training sequence for all instances
path = '/content/kaggle/train/'
sequences = list()

for i in range(0, len(listdir(path))):
    file_name = path + str(i) + '.npy'
    data = np.load(file_name)
    df = pd.DataFrame(data=data[:,:], index=None, columns=None)
    df = df.drop(df.iloc[:,64:92].head(0).columns, axis=1)
    values = df.values
    sequences.append(values)

In [0]:
# read the label
target = pd.read_csv('/content/kaggle/train_kaggle.csv')
target = target.values[:,1]

In [0]:
# # preprocessing steps
# len_sequences = []

# for seq in sequences:
#     len_sequences.append(len(seq))
    
# pd.Series(len_sequences).describe()


In [0]:
# x = np.array(len_sequences)
# print(np.unique(x, return_counts=True))

In [0]:
# # 90% quartile is 1000 length, so the rest need to pad up to 1000 length
# pd.Series(len_sequences).quantile(q=0.9)

In [0]:
'''
# dont pad
final_seq = np.array(sequences)
final_seq.shape
final_seq[1].shape
'''

'\n# dont pad\nfinal_seq = np.array(sequences)\nfinal_seq.shape\nfinal_seq[1].shape\n'

In [0]:
# padding the sequence with values from last row for each sequence
# since most of the sequence are 1000 length, no need to truncate further after padding

to_pad = 1000
new_sequences = []
for seq in sequences:
    len_seq = len(seq)
    # last_val = seq[-1]
    last_val = [0]*74
    n = to_pad - len_seq
    '''
    to_concat = np.repeat(seq[-1], n).reshape(102, n).transpose()
    new_seq = np.concatenate([seq, to_concat])
    new_sequences.append(new_seq)
    '''
    to_concat = np.repeat(last_val, n).reshape(74, n).transpose()
    new_seq = np.concatenate([seq, to_concat])
    new_sequences.append(new_seq)
    
final_seq = np.stack(new_sequences)

In [0]:
'''
#resize all, pad 0s
new_sequences = []

for seq in sequences:
  new_seq = seq.copy()
  new_seq.resize((1000,102), refcheck=False)
  new_sequences.append(new_seq)

final_seq = np.stack(new_sequences)
'''

'\n#resize all, pad 0s\nnew_sequences = []\n\nfor seq in sequences:\n  new_seq = seq.copy()\n  new_seq.resize((1000,102), refcheck=False)\n  new_sequences.append(new_seq)\n\nfinal_seq = np.stack(new_sequences)\n'

In [0]:
# #check that sequences are padded to 0 
# new_sequences[11][57]

In [0]:
#clear memory 
import gc
sequences = None
len_sequences = None
new_sequences = None
history = None
predict_data = None
model = None
visible = None
left_conv1 = None
right_conv1 = None
merge = None
gru = None
output = None
gc.collect()

0

In [0]:
# split the data into training, test, validation
from sklearn.model_selection import train_test_split, StratifiedKFold

'''
X_train, X_test, y_train, y_test = train_test_split(final_seq, target, test_size=0.2, random_state=1)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
X_valid = np.array(X_valid)
y_valid = np.array(y_valid)
'''

#k fold CV
k=10
folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=None).split(final_seq, target))

In [0]:
'''
#Check results of k-fold split
for j, (train_idx, val_idx) in enumerate(folds):
    print('\nFold ',j)
    print('train_idx = ',train_idx)
    print('val_idx = ',val_idx)
    X_train_cv = final_seq[train_idx]
    y_train_cv = target[train_idx]
    X_valid_cv = final_seq[val_idx]
    y_valid_cv = target[val_idx]

    # print('X_train_cv shape:', X_train_cv.shape)
    # print('y_train_cv shape:', y_train_cv.shape)
    # print('X_valid_cv shape:', X_valid_cv.shape)
    # print('y_valid_cv shape:', y_valid_cv.shape)
'''

"\n#Check results of k-fold split\nfor j, (train_idx, val_idx) in enumerate(folds):\n    print('\nFold ',j)\n    print('train_idx = ',train_idx)\n    print('val_idx = ',val_idx)\n    X_train_cv = final_seq[train_idx]\n    y_train_cv = target[train_idx]\n    X_valid_cv = final_seq[val_idx]\n    y_valid_cv = target[val_idx]\n\n    # print('X_train_cv shape:', X_train_cv.shape)\n    # print('y_train_cv shape:', y_train_cv.shape)\n    # print('X_valid_cv shape:', X_valid_cv.shape)\n    # print('y_valid_cv shape:', y_valid_cv.shape)\n"

# Checkpoint: Environment Initialised

In [0]:
print("Environment Initialised Sucessfully")

Environment Initialised Sucessfully


# Define model and train

## 1_Best Model
(added maxpooling/averagepooling layer, regularization)

In [0]:
model_id = 1

backend.clear_session()

#input layer
visible = Input(shape=(1000,74))

#2 separate convolution1 layers
left_conv1 = Conv1D(256, kernel_size=4, strides=2, padding='same')(visible)
right_conv1 = Conv1D(256, kernel_size=4, strides=2, padding='same', activation='sigmoid')(visible)
merge = concatenate([left_conv1, right_conv1])
gru = GRU(128, dropout=0.3, recurrent_dropout=0.3, 
          activation="sigmoid",recurrent_activation="sigmoid")(merge)
          # activity_regularizer=regularizers.l1(0.01))(merge)
output = Dense(1, activation='sigmoid')(gru)
model = Model(inputs=[visible], outputs=output)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000, 74)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 500, 256)     76032       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 500, 256)     76032       input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 500, 512)     0           conv1d_1[0][0]                   
                                                                 conv1d_2[0][0]             

## 2_ Add additional GRU, reduce feature maps sizes
Kaggle score only 0.982. Bad

In [0]:
model_id = 2

backend.clear_session()

#input layer
visible = Input(shape=(1000,74))

#2 separate convolution1 layers
left_conv1 = Conv1D(128, kernel_size=4, strides=2, padding='same')(visible)
right_conv1 = Conv1D(128, kernel_size=4, strides=2, padding='same', activation='sigmoid')(visible)
merge = concatenate([left_conv1, right_conv1])
gru = GRU(64, dropout=0.3, recurrent_dropout=0.3, activation="sigmoid",recurrent_activation="sigmoid", return_sequences=True)(merge)
gru2 = GRU(64, dropout=0.3, recurrent_dropout=0.3, activation="sigmoid",recurrent_activation="sigmoid")(gru)
output = Dense(1, activation='sigmoid')(gru2)
model = Model(inputs=[visible], outputs=output)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000, 74)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 500, 128)     38016       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 500, 128)     38016       input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 500, 256)     0           conv1d_1[0][0]                   
                                                                 conv1d_2[0][0]             

## 3_Bidirectional GRU 2nd one (score 0.98, model=3)

In [0]:
from keras.layers import Bidirectional

model_id = 3

backend.clear_session()

#input layer
visible = Input(shape=(1000,74))

#2 separate convolution1 layers
left_conv1 = Conv1D(256, kernel_size=4, strides=2, padding='same')(visible)
right_conv1 = Conv1D(256, kernel_size=4, strides=2, padding='same', activation='sigmoid')(visible)
merge = concatenate([left_conv1, right_conv1])
gru = Bidirectional(GRU(128, dropout=0.3, recurrent_dropout=0.3, 
          activation="sigmoid",recurrent_activation="sigmoid"))(merge)
          # activity_regularizer=regularizers.l1(0.01)))(merge)
output = Dense(1, activation='sigmoid')(gru)
model = Model(inputs=[visible], outputs=output)

model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000, 74)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 500, 256)     76032       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 500, 256)     76032       input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 500, 512)     0           conv1d

## from andy

In [0]:
backend.clear_session()

model = Sequential() 
model.add(Conv1D(128, input_shape=(1000, 74), activation='relu', kernel_size=4, strides=2, padding='same')) 
model.add(GRU(128, dropout=0.2, activation='hard_sigmoid', recurrent_activation='hard_sigmoid', input_shape=(1000, 74))) 
model.add(Dense(1, activation='sigmoid'))
model.summary()

### Model 0

In [0]:
backend.clear_session()

model = Sequential() 
model.add(Conv1D(512, input_shape=(None, 102), kernel_size=2, strides=2, padding='same', use_bias=True)) 
model.add(MaxPooling1D(pool_size=2, strides=2)) 
model.add(GRU(512, dropout=0.3, return_sequences=True, use_bias=True))
model.add(GRU(256, dropout=0.2,return_sequences=True, use_bias=True))
model.add(Conv1D(256, kernel_size=2, strides=2, padding='same', use_bias=True)) 
model.add(MaxPooling1D(pool_size=2, strides=2)) 
model.add(GRU(256, dropout=0.3, return_sequences=True, use_bias=True)) 
model.add(GRU(128, dropout=0.2, use_bias=True)) 
# model.add(GlobalAvgPooling)      try later
# model.add(Dropout(0.3))
# model.add(Dense(64, activation="sigmoid", use_bias=True))
model.add(Dense(1, activation="sigmoid", use_bias=True))
# model.add(Softmax layer)   try later. from lecnotes

model.summary()

## Other tries

### With embeddings

In [0]:
backend.clear_session()

model = Sequential() 
model.add(Embedding(10000, 64, input_length=102))
model.add(Conv1D(256, input_shape=(None, 74), kernel_size=2, strides=2, padding='same')) 
model.add(MaxPooling1D(pool_size=2, strides=2)) 
model.add(Bidirectional(GRU(256, dropout=0.3, return_sequences=True, input_shape=(1000, 74))))
model.add(Bidirectional(GRU(128,dropout=0.2,return_sequences=True)))
model.add(Conv1D(256, input_shape=(1000, 74), kernel_size=2, strides=2, padding='same')) 
model.add(MaxPooling1D(pool_size=2, strides=2)) 
model.add(GRU(256, dropout=0.2, return_sequences=True, input_shape=(1000, 74))) 
model.add(GRU(128)) 
model.add(Dense(1, activation="sigmoid"))

model.summary()

### Sequential Model 1
Conv1D -> MaxPool -> GRUx2 -> Densex2

In [0]:
backend.clear_session()

# Build time series classification model
model = Sequential()
model.add(Conv1D(256, input_shape=(1000, 102), kernel_size=2, strides=2, padding='same'))
# model.add(Conv1D(128, kernel_size=2, strides=1, padding='valid'))
model.add(MaxPooling1D(pool_size=2, strides=2))
model.add(GRU(256, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(128, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(32, activation="sigmoid"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.summary()

### Sequential Model 2
Conv1D -> Conv1D -> MaxPool -> Bidirection(GRU)x2 -> Densex2

In [0]:
backend.clear_session()

# Build time series classification model
model = Sequential()
model.add(Conv1D(256, input_shape=(1000, 102), kernel_size=2, strides=2, padding='same'))
model.add(MaxPooling1D(pool_size=2, strides=2))
model.add(Conv1D(128, kernel_size=2, strides=1, padding='valid'))
model.add(MaxPooling1D(pool_size=2, strides=1))
model.add(Bidirectional(GRU(256, dropout=0.3, recurrent_dropout=0.3, return_sequences=True)))
# model.add(Dropout(0.5))
model.add(Bidirectional(GRU(128, dropout=0.3, recurrent_dropout=0.3, )))
model.add(Dense(32, activation="sigmoid"))
model.add(Dropout(0.4))
model.add(Dense(1, activation="sigmoid"))

model.summary()

### Sequential Model 3

In [0]:
backend.clear_session()

model = Sequential() 
model.add(Conv1D(256, input_shape=(1000, 102), kernel_size=2, strides=2, padding='same')) 
model.add(MaxPooling1D(pool_size=2, strides=2)) 
model.add(GRU(256, dropout=0.3, return_sequences=True, input_shape=(1000, 74))) 
model.add(GRU(128,dropout=0.2,return_sequences=True)) 
model.add(Conv1D(256, input_shape=(1000, 74), kernel_size=2, strides=2, padding='same')) 
model.add(MaxPooling1D(pool_size=2, strides=2)) 
model.add(GRU(256, dropout=0.2, return_sequences=True, input_shape=(1000, 74))) 
model.add(GRU(128)) 
model.add(Dense(1, activation="sigmoid")) 

model.summary()

### Sequential 4 (02Nov2019)

In [0]:
backend.clear_session()

model = Sequential()
# Prevent overfitting using dropout method of regularization
model.add(Dropout(0.5))
model.add(LSTM(64, input_shape=(1000, 102), recurrent_dropout=0.5))
model.add(Dropout(0.5))
# Condense to single binary output value
model.add(Dense(1, activation='sigmoid'))

# model.summary()

### Sequential 5 (02Nov 2019)

In [0]:
backend.clear_session()

model = Sequential() 
model.add(Conv1D(256, input_shape=(None, 74), kernel_size=2, strides=2, padding='same')) 
model.add(MaxPooling1D(pool_size=2, strides=2)) 
model.add(LSTM(256, dropout=0.3, return_sequences=True, use_bias=True, input_shape=(1000, 74)))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

model.summary()

### Functional Model

In [0]:
from keras.layers import Input, Flatten, ReLU
from keras.layers.merge import concatenate
from keras.models import Model

backend.clear_session()

#input layer
visible = Input(shape=(1000,102))
conv1 = Conv1D(256, input_shape=(1000, 102), kernel_size=2, strides=2, padding='same')(visible)
pool1 = MaxPooling1D(pool_size=2, strides=2)(conv1)
gru1 = GRU(256, dropout=0.3, recurrent_dropout=0.3, return_sequences=True)(pool1)
gru2 = GRU(128,dropout=0.2, recurrent_dropout=0.3)(gru1)
dense1 = Dense(64, activation='sigmoid')(gru2)

dense2 = Dense(32, activation='sigmoid')(dense1)
dense3 = Dense(32, activation='sigmoid')(dense1)
dense4 = Dense(10, activation='sigmoid')(dense3)

merge = concatenate([dense2, dense4])
output = Dense(1, activation='sigmoid')(merge)

'''
#2 separate convolution layers
left_conv = Conv1D(128, kernel_size=2, strides=2, padding='same')(visible)
right_conv = Conv1D(128, kernel_size=2, strides=2, padding='same', activation='sigmoid')(visible)

left_pool = MaxPooling1D(pool_size=2, strides=2)(left_conv)
right_pool = MaxPooling1D(pool_size=2, strides=2)(right_conv)

left_gru = GRU(256, dropout=0.3, recurrent_dropout=0.3)(left_pool) 
right_gru = GRU(256, dropout=0.3, recurrent_dropout=0.3)(left_pool) 

#left_flat = Flatten()(left_pool)
#right_flat = Flatten()(right_pool)

#merge convolution layers
merge = concatenate([left_gru, right_gru])
relu = ReLU()(merge)
# pool = MaxPooling1D(pool_size=2, strides=2)(relu)

# gru1 = GRU(256, dropout=0.2, return_sequences=True)(pool) 
# gru2 = GRU(128, dropout=0.2)(gru1)

# flat = Flatten()(pool)
dense1 = Dense(128, activation='relu')(relu)
output = Dense(1, activation='softmax')(dense1)

'''

model = Model(inputs=[visible], outputs=output)
model.summary()

## Define callbacks and custom functions

### Custom function - Classes

In [0]:
num_epoch = 10

class XTensorBoard(TensorBoard):
    def on_epoch_end(self, epoch, logs=None):
      print('Learning rate = ', backend.eval(self.model.optimizer.lr))

class OptimizerChanger(EarlyStopping):
    def __init__(self, on_train_end, **kwargs):
        self.do_on_train_end = on_train_end
        super(OptimizerChanger,self).__init__(**kwargs)

    def on_train_end(self, logs=None):
        # super(OptimizerChanger,self).on_train_end(self,logs)
        print('Switching to SGD Optimizer')
        ## TODO ##
        # reset the learning rate for SGD to 0.0005
        super(OptimizerChanger,self).on_train_end(logs)
        self.do_on_train_end()


### Create Callbacks

In [0]:
# sgd = SGD(lr=0.0005, decay=1e-6, momentum=0.9, nesterov=True)

# 1. print learning rate
cb_print_lr = XTensorBoard()

# 2. define callback - reduce learning rate on plateau
cb_reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2, min_lr=0.000001)

# 3. define callback - early stopping
cb_early_stopping = EarlyStopping(monitor='val_acc', min_delta=0.001, patience=4, restore_best_weights=True)

# 4. define callback - model checkpoint
filepath = "/content/drive/My Drive/NUS_2019/cs5242_kaggle/models/"+str(model_id)+"_model-{epoch:02d}-{val_acc:.4f}.hdf5"
cb_checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callback_op_change = [cb_print_lr, cb_reduce_lr, cb_early_stopping, cb_checkpoint]

# 5. define callback - change to SGD if acc does not improve
def do_after_training():
    #warining, this creates a new optimizer and,
    #at the beginning, it might give you a worse training performance than before
    adam = Adam(lr=0.0005)
    model.compile(optimizer = adam, loss='binary_crossentropy', metrics=['acc'])
    model.fit(X_train_cv, y_train_cv, 
                        epochs=1,
                        batch_size=64,
                        validation_data=(X_valid_cv, y_valid_cv), 
                        callbacks=callback_op_change)
    
changer = OptimizerChanger(on_train_end=do_after_training, 
                           monitor='val_acc',
                           min_delta=0.005,
                           patience=1)

# define callback - history
history = History()

# cbks = [history, cb_print_lr, cb_reduce_lr, cb_checkpoint, changer]
cbks = [history, cb_print_lr, cb_reduce_lr, cb_checkpoint, cb_early_stopping]

# define callback - learning rate scheduler (replaced with reduceLRonPlateau)
# reducing the learning rate by half every 2 epochs
# cbks = [LearningRateScheduler(lambda epoch: 0.001 * 0.5 ** (epoch // 2)),
#         TensorBoard(write_graph=False)]





# Compile and Train

## Train with k fold CV (batchsize = 128)

In [0]:
for j, (train_idx, val_idx) in enumerate(folds):
    print('\nFold ',j)
    
    X_train_cv = final_seq[train_idx]
    y_train_cv = target[train_idx]
    X_valid_cv = final_seq[val_idx]
    y_valid_cv = target[val_idx]

    # print_lr = MyCallBack()
    
    cbks = [history, cb_print_lr, cb_reduce_lr, cb_checkpoint, cb_early_stopping]
    # train the model
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['acc'])

    #define early stopping to stop when validation is not decreasing anymore
    # early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.005, patience=5, restore_best_weights=True)

    ## TODO ##
    # 1. Decrease batch size for Adam every n epochs
    # 2. Decrease learning rate for Adam every n epochs
    # 3. Reset learning rate for SGD every n epochs / or just change to Adam

    history = model.fit(X_train_cv, y_train_cv, 
                        epochs=2,
                        batch_size=128,
                        validation_data=(X_valid_cv, y_valid_cv), 
                        callbacks=cbks)

    # print(model.evaluate(X_valid_cv, y_valid_cv))



Fold  0


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 16795 samples, validate on 1867 samples







Epoch 1/2
16795/16795 [==============================] - 288s 17ms/step - loss: 0.3640 - acc: 0.8326 - val_loss: 0.2749 - val_acc: 0.8784
Learning rate =  0.001

Epoch 00001: val_acc improved from -inf to 0.87841, saving model to /content/drive/My Drive/NUS_2019/cs5242_kaggle/models/3_model-01-0.8784.hdf5
Epoch 2/2
16795/16795 [==============================] - 288s 17ms/step - loss: 0.2543 - acc: 0.8950 - val_loss: 0.2207 - val_acc: 0.9138
Learning rate =  0.001

Epoch 00002: val_acc improved from 0.87841 to 0.91377, saving model to /content/drive/My Drive/NUS_2019/cs5242_kaggle/models/3_model-02-0.9138.hdf5

Fold  1
Train on 16795 samples, validate on 1867 samples
Epoch 1/2
16795/16795 [==============================] - 291s 17ms/step - loss: 0.2101 - acc: 0.9166 - val_loss: 0.1885 - val_acc: 0.9250
Learning rate =  0.001


In [0]:
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import confusion_matrix

for j, (train_idx, val_idx) in enumerate(folds):
    print('\nFold ',j)
    
    X_train_cv = final_seq[train_idx]
    y_train_cv = target[train_idx]
    X_valid_cv = final_seq[val_idx]
    y_valid_cv = target[val_idx]
    
    predict_data = model.predict(X_valid_cv, batch_size=128)

    print('Accuracy Score : ' + str(accuracy_score(y_valid_cv,predict_data.round())))
    print('Precision Score : ' + str(precision_score(y_valid_cv,predict_data.round())))
    print('Recall Score : ' + str(recall_score(y_valid_cv,predict_data.round())))
    print('F1 Score : ' + str(f1_score(y_valid_cv,predict_data.round())))

    #Dummy Classifier Confusion matrix
    print('Confusion Matrix : \n' + str(confusion_matrix(y_valid_cv,predict_data.round())))

    break


Fold  0
Accuracy Score : 0.5634708087841457
Precision Score : 0.5634708087841457
Recall Score : 1.0
F1 Score : 0.7207947927372388
Confusion Matrix : 
[[   0  815]
 [   0 1052]]


## Reduce batch size and train

In [0]:
for j, (train_idx, val_idx) in enumerate(folds):
    print('\nFold ',j)
    
    #reset optmizers with each k
    # adam = Adam(lr=0.0005)
    # sgd = SGD(lr=0.0005, momentum=0.8)

    # callbacks_1 = [history, cb_print_lr, cb_reduce_lr, cb_checkpoint, changer]

    X_train_cv = final_seq[train_idx]
    y_train_cv = target[train_idx]
    X_valid_cv = final_seq[val_idx]
    y_valid_cv = target[val_idx]

    # print_lr = MyCallBack()
    cbks = [history, cb_print_lr, cb_reduce_lr, cb_checkpoint, changer]

    # train the model
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['acc'])

    #define early stopping to stop when validation is not decreasing anymore
    # early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.005, patience=5, restore_best_weights=True)

    ## TODO ##
    # 1. Decrease batch size for Adam every n epochs
    # 2. Decrease learning rate for Adam every n epochs
    # 3. Reset learning rate for SGD every n epochs / or just change to Adam

    history = model.fit(X_train_cv, y_train_cv, 
                        epochs=1,
                        batch_size=128,
                        validation_data=(X_valid_cv, y_valid_cv), 
                        callbacks=cbks)

    # print(model.evaluate(X_valid_cv, y_valid_cv))



Fold  0
Train on 16795 samples, validate on 1867 samples
Epoch 1/1
16795/16795 [==============================] - 291s 17ms/step - loss: 0.0898 - acc: 0.9684 - val_loss: 0.0905 - val_acc: 0.9630
Learning rate =  0.001

Epoch 00001: val_acc did not improve from 0.97374
Switching to SGD Optimizer
Train on 16795 samples, validate on 1867 samples
Epoch 1/1
16795/16795 [==============================] - 425s 25ms/step - loss: 0.0821 - acc: 0.9713 - val_loss: 0.0753 - val_acc: 0.9721
Learning rate =  0.0005

Epoch 00001: val_acc did not improve from 0.97374

Fold  1
Train on 16795 samples, validate on 1867 samples
Epoch 1/1
16795/16795 [==============================] - 288s 17ms/step - loss: 0.0857 - acc: 0.9686 - val_loss: 0.0728 - val_acc: 0.9770
Learning rate =  0.001

Epoch 00001: val_acc improved from 0.97374 to 0.97697, saving model to /content/drive/My Drive/NUS_2019/cs5242_kaggle/models/3_model-01-0.9770.hdf5
Switching to SGD Optimizer
Train on 16795 samples, validate on 1867 sampl

KeyboardInterrupt: ignored

In [0]:
print(model.metrics_names)

['loss', 'acc']


## Train without k-fold CV

In [0]:
'''
# train the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

#define early stopping to stop when validation is not decreasing anymore
# early_stopping = EarlyStopping(monitor='val_acc', min_delta=0.005, patience=20, restore_best_weights=True)

history = model.fit(X_train, y_train, 
                    epochs=100,
                    batch_size=128,
                    validation_data=(X_valid, y_valid))
                    # , 
                    # callbacks=[early_stopping])
'''

# Results and Test

## Load best model from Google Drive

In [0]:
model_path = '/content/drive/My Drive/NUS_2019/cs5242_kaggle/models/'
model_fn = 'run_20191107/2_model-02-0.9866.hdf5'

model.compile(optimizer='Adam',
                  loss='binary_crossentropy',
                   metrics=['acc'])

model.load_weights(model_path+model_fn)
print("Model Loaded")

Model Loaded


## Get predictions for Test data (count=6501)

In [0]:
# read all training sequence for all instances
path = '/content/kaggle/test/'
sequences = list()

for i in range(0, len(listdir(path))):
    file_name = path + str(i) + '.npy'
    data = np.load(file_name)
    df = pd.DataFrame(data=data[:,:], index=None, columns=None)
    df = df.drop(df.iloc[:,64:92].head(0).columns, axis=1)
    values = df.values
    sequences.append(values)

In [0]:
# padding the sequence with values from last row for each sequence
# since most of the sequence are 1000 length, no need to truncate further after padding

to_pad = 1000
new_sequences = []
for seq in sequences:
    len_seq = len(seq)
    # last_val = seq[-1]
    last_val = [0]*74
    n = to_pad - len_seq
  
    '''
    to_concat = np.repeat(seq[-1], n).reshape(102, n).transpose()
    new_seq = np.concatenate([seq, to_concat])
    new_sequences.append(new_seq)
    '''
    to_concat = np.repeat(last_val, n).reshape(74, n).transpose()
    new_seq = np.concatenate([seq, to_concat])
    new_sequences.append(new_seq)
    
final_seq_test = np.stack(new_sequences)

In [0]:
#save the model, with the validation score and timestamp
now = datetime.now(timezone('Asia/Singapore'))
curr_timestamp = now.strftime("%Y%m%d_%H%M")

test_data = np.array(final_seq_test)
predict_data = model.predict(test_data, batch_size=128)

#save solutions file to Google Drive
sol_path = '/content/drive/My Drive/NUS_2019/cs5242_kaggle/models/solution_' + curr_timestamp + '.csv'
predict_df = pd.DataFrame(data=np.array(predict_data), columns=['Predicted']).to_csv(sol_path, index_label='Id')

In [0]:
predict_df_1 = pd.DataFrame(data=np.array(predict_data), columns=['Predicted'])

In [0]:
predict_df_1['Predicted_Round'] = predict_df_1['Predicted'].round(3)

In [0]:
predict_df_1.sample(5)

,Predicted,Predicted_Round
3384,0.965190,0.965
433,0.994970,0.995
4815,0.999993,1.000
2361,0.999993,1.000
4440,0.321670,0.322
